In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from os import listdir

import tensorflow as tf

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
beginner_path = 'beginner_expert_processedData/beginner/beginner_'
expert_path = 'beginner_expert_processedData/expert/expert_'
left_column = [
#'Time',
    'Distance','GPS Latitude','GPS Longitude','Damper Velocity (Calc) FL','Damper Velocity (Calc) FR','Damper Velocity (Calc) RL',
'Damper Velocity (Calc) RR','Corr Dist','Corr Dist (Unstretched)','Corr Speed','Brake Pos',
'CG Accel Lateral','CG Accel Longitudinal','CG Accel Vertical','CG Height','Camber FL','Camber FR','Camber RL','Camber RR','Car Coord X',
'Car Coord Y','Car Coord Z','Car Pos Norm','Chassis Pitch Angle','Chassis Pitch Rate','Chassis Roll Angle','Chassis Roll Rate',
'Chassis Velocity X','Chassis Velocity Y','Chassis Velocity Z','Chassis Yaw Rate','Drive Train Speed','Engine RPM','Ground Speed',
'Ride Height FL','Ride Height FR','Ride Height RL','Ride Height RR','Road Temp','Self Align Torque FL','Self Align Torque FR',
'Self Align Torque RL','Self Align Torque RR','Session Time Left','Steering Angle','Suspension Travel FL','Suspension Travel FR',
'Suspension Travel RL','Suspension Travel RR','Tire Load FL','Tire Load FR','Tire Load RL','Tire Load RR','Tire Loaded Radius FL',
'Tire Loaded Radius FR','Tire Loaded Radius RL','Tire Loaded Radius RR','Tire Pressure FL','Tire Pressure FR','Tire Pressure RL','Tire Pressure RR',
'Tire Rubber Grip FL','Tire Rubber Grip FR','Tire Rubber Grip RL','Tire Rubber Grip RR','Tire Slip Angle FL','Tire Slip Angle FR',
'Tire Slip Angle RL','Tire Slip Angle RR','Tire Slip Ratio FL','Tire Slip Ratio FR','Tire Slip Ratio RL','Tire Slip Ratio RR',
'Tire Temp Core FL','Tire Temp Core FR','Tire Temp Core RL','Tire Temp Core RR','Tire Temp Inner FL','Tire Temp Inner FR',
'Tire Temp Inner RL','Tire Temp Inner RR','Tire Temp Middle FL','Tire Temp Middle FR','Tire Temp Middle RL',
'Tire Temp Middle RR','Tire Temp Outer FL','Tire Temp Outer FR','Tire Temp Outer RL','Tire Temp Outer RR','Toe In FL',
'Toe In FR','Toe In RL','Toe In RR','Wheel Angular Speed FL','Wheel Angular Speed FR','Wheel Angular Speed RL','Wheel Angular Speed RR',
'CG Distance','Lateral Velocity','Longitudinal Velocity','Lateral Acceleration','Longitudinal Acceleration']

curveList = [[103.9, 209.3], [316.6, 399.6], [425.3, 517.9], [590.5, 756.9], [1048.7, 1110.5], [1212.3, 1437.1]]
curve_num = 0
sequences = list()
for i in range(1,20):
    file_path = beginner_path + str(i) + '_new2.csv'
    print(file_path)
    df = pd.read_csv(file_path, header=0)
    df = df.dropna()
    df = df.astype(float)
    df = df[(df['Distance'] >= curveList[curve_num][0]) & (df['Distance'] <= curveList[curve_num][1])]
    df = df.loc[:,left_column]

    values = df.values
    sequences.append(values)
    
for i in range(1,20):
    file_path = expert_path + str(i) + '_new2.csv'
    print(file_path)
    df = pd.read_csv(file_path, header=0)
    df = df.dropna()
    df = df.astype(float)
    df = df[(df['Distance'] >= curveList[curve_num][0]) & (df['Distance'] <= curveList[curve_num][1])]
    df = df.loc[:,left_column]

    values = df.values
    sequences.append(values)
    

beginner_expert_processedData/beginner/beginner_1_new2.csv
beginner_expert_processedData/beginner/beginner_2_new2.csv
beginner_expert_processedData/beginner/beginner_3_new2.csv
beginner_expert_processedData/beginner/beginner_4_new2.csv
beginner_expert_processedData/beginner/beginner_5_new2.csv
beginner_expert_processedData/beginner/beginner_6_new2.csv
beginner_expert_processedData/beginner/beginner_7_new2.csv
beginner_expert_processedData/beginner/beginner_8_new2.csv
beginner_expert_processedData/beginner/beginner_9_new2.csv
beginner_expert_processedData/beginner/beginner_10_new2.csv
beginner_expert_processedData/beginner/beginner_11_new2.csv
beginner_expert_processedData/beginner/beginner_12_new2.csv
beginner_expert_processedData/beginner/beginner_13_new2.csv
beginner_expert_processedData/beginner/beginner_14_new2.csv
beginner_expert_processedData/beginner/beginner_15_new2.csv
beginner_expert_processedData/beginner/beginner_16_new2.csv
beginner_expert_processedData/beginner/beginner_1

In [3]:
print(len(sequences))
print(sequences[0].shape)

38
(72, 102)


In [4]:
targets = pd.read_csv('target_19.csv')
targets = targets.values[:,1]

In [5]:
len_sequences = []
for one_seq in sequences:
    len_sequences.append(len(one_seq))
    
print(len_sequences)
pd.Series(len_sequences).describe()

[72, 71, 78, 91, 68, 69, 75, 84, 76, 79, 97, 102, 101, 89, 63, 74, 80, 69, 78, 68, 64, 64, 71, 68, 65, 66, 66, 67, 66, 72, 66, 67, 70, 68, 62, 68, 66, 72]


count     38.000000
mean      73.473684
std       10.376004
min       62.000000
25%       66.250000
50%       69.500000
75%       77.500000
max      102.000000
dtype: float64

In [6]:
#Padding the sequence with the values in last row to max length
to_pad = 120
new_seq = []
for one_seq in sequences:
    len_one_seq = len(one_seq)
    last_val = one_seq[-1]
    n = to_pad - len_one_seq
   
    to_concat = np.repeat(one_seq[-1], n).reshape(102, n).transpose()
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)
final_seq = np.stack(new_seq)

#truncate the sequence to length 60
from tensorflow.keras.preprocessing import sequence
seq_len = 60
final_seq=sequence.pad_sequences(final_seq, maxlen=seq_len, padding='post', dtype='float', truncating='post')


In [7]:
groups = pd.read_csv('group_19.csv', header=0)
groups = groups.values[:,1]

In [8]:
train = [final_seq[i] for i in range(len(groups)) if (groups[i]==1)]
validation = [final_seq[i] for i in range(len(groups)) if groups[i]==2]
test = [final_seq[i] for i in range(len(groups)) if groups[i]==3]

train_target = [targets[i] for i in range(len(groups)) if (groups[i]==1)]
validation_target = [targets[i] for i in range(len(groups)) if groups[i]==2]
test_target = [targets[i] for i in range(len(groups)) if groups[i]==3]

# train
test_target
test

[array([[105.        ,  44.05372833, -78.6817109 , ...,  21.60610265,
          -7.20203456,  46.81322263],
        [106.7       ,  44.05371304, -78.68170802, ...,  23.0465095 ,
         -10.80305098,  46.81322263],
        [108.4       ,  44.05369866, -78.68170515, ...,  26.28742491,
          18.00508554,  61.2172906 ],
        ...,
        [192.3       ,  44.05317426, -78.68121915, ..., 101.54868247,
         -39.61118835,  25.20711953],
        [193.7       ,  44.05317183, -78.68120611, ...,  98.66786876,
         -54.01525661, -90.02542769],
        [195.        ,  44.05316994, -78.68119451, ...,  92.5461397 ,
         -50.41423934, -36.01017108]]),
 array([[105.3       ,  44.05371098, -78.68168095, ...,  35.65006938,
         -90.02542797, -46.81322206],
        [107.1       ,  44.05369551, -78.68167673, ...,  33.84956082,
         -10.80305127, -21.60610253],
        [108.9       ,  44.05367932, -78.6816725 , ...,  33.48945912,
         -14.40406854, -14.40406854],
        ...,


In [9]:
train = np.array(train)
validation = np.array(validation)
test = np.array(test)

train_target = np.array(train_target)
validation_target = np.array(validation_target)
test_target = np.array(test_target)

print(train.shape)
print(train_target.shape)

(30, 60, 102)
(30,)


In [10]:
model = Sequential()
model.add(LSTM(256, input_shape=(seq_len, 102)))
model.add(Dense(1, activation='sigmoid'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 256)               367616    
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 367,873
Trainable params: 367,873
Non-trainable params: 0
_________________________________________________________________


In [12]:
adam = Adam(lr=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(train, train_target, epochs=50, batch_size=256, callbacks=[chk], validation_data=(validation,validation_target))

Epoch 1/50


c:\users\ssdkms\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


1/1 [==============================] - 1s 1s/step - loss: 0.8778 - accuracy: 0.3000
Epoch 2/50
1/1 [==============================] - 0s 97ms/step - loss: 0.7760 - accuracy: 0.4000
Epoch 3/50
1/1 [==============================] - 0s 97ms/step - loss: 0.6929 - accuracy: 0.6333
Epoch 4/50
1/1 [==============================] - 0s 95ms/step - loss: 0.6329 - accuracy: 0.7000
Epoch 5/50
1/1 [==============================] - 0s 95ms/step - loss: 0.6050 - accuracy: 0.7000
Epoch 6/50
1/1 [==============================] - 0s 93ms/step - loss: 0.5735 - accuracy: 0.8333
Epoch 7/50
1/1 [==============================] - 0s 93ms/step - loss: 0.5479 - accuracy: 0.8667
Epoch 8/50
1/1 [==============================] - 0s 96ms/step - loss: 0.5079 - accuracy: 0.8667
Epoch 9/50
1/1 [==============================] - 0s 97ms/step - loss: 0.4901 - accuracy: 0.8667
Epoch 10/50
1/1 [==============================] - 0s 95ms/step - loss: 0.4701 - accuracy: 0.8333
Epoch 11/50
1/1 [=========================

Epoch 48/50
1/1 [==============================] - 0s 93ms/step - loss: 0.2344 - accuracy: 0.9667
Epoch 49/50
1/1 [==============================] - 0s 94ms/step - loss: 0.2329 - accuracy: 0.9667
Epoch 50/50
1/1 [==============================] - 0s 94ms/step - loss: 0.2296 - accuracy: 0.9667


In [13]:
model.evaluate(test,test_target)

1/1 [==============================] - 0s 447ms/step - loss: 0.3998 - accuracy: 0.7500


[0.39976686239242554, 0.75]